In [1]:
import tensorflow as tf
import numpy as np
import os
import pandas as pd

2023-07-06 15:21:50.679704: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [1]:

current_directory = os.listdir()
match = [match for match in current_directory if 'for_modeling' in match]
match = match[0]

pd.set_option('display.max_rows', 500)

df = pd.read_csv(match, index_col=0)

2023-07-05 14:04:34.071430: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99178 entries, 0 to 99177
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Date               99178 non-null  object 
 1   LittleFallsGage    99174 non-null  float64
 2   01646500-00060     99174 non-null  float64
 3   SenecaGage         99178 non-null  float64
 4   01645000-00060     99178 non-null  float64
 5   PointofRocksGage   99178 non-null  float64
 6   01638500-00060     99178 non-null  int64  
 7   EdwardsGage        99178 non-null  float64
 8   ShhepardstownGage  99177 non-null  float64
 9   01618000-00060     99177 non-null  float64
 10  HancockGage        99178 non-null  float64
 11  01613000-00060     99178 non-null  int64  
 12  SpringfieldGage    99178 non-null  float64
 13  01608500-00060     99178 non-null  int64  
dtypes: float64(10), int64(3), object(1)
memory usage: 11.4+ MB


In [3]:
# lag variables
# positive lags shift obs down, so they push past, old obs down so they are next to current obs
df['PointofRocksGage'] = df['PointofRocksGage'].shift(16)
df['EdwardsGage'] = df['EdwardsGage'].shift(12)
df['ShhepardstownGage'] = df['ShhepardstownGage'].shift(32)
df['HancockGage'] = df['HancockGage'].shift(60)
df['SpringfieldGage'] = df['SpringfieldGage'].shift(68)

# to add more lagged fetures for 17hr prediction
df['SpringfieldGage_extrashift1'] = df['SpringfieldGage'].shift(1)
df['SpringfieldGage_extrashift2'] = df['SpringfieldGage'].shift(2)
df['SpringfieldGage_extrashift3'] = df['SpringfieldGage'].shift(3)

df.drop(df.columns[[2,4,6,9,11,13]], axis=1, inplace=True)

In [4]:
df = df.dropna()

In [5]:
df.head(20)

,Date,LittleFallsGage,SenecaGage,PointofRocksGage,EdwardsGage,ShhepardstownGage,HancockGage,SpringfieldGage,SpringfieldGage_extrashift1,SpringfieldGage_extrashift2,SpringfieldGage_extrashift3
71,2022-04-13 07:00:00,4.99,2.35,5.26,7.88,5.85,7.45,3.96,3.97,3.97,3.98
72,2022-04-13 07:15:00,4.98,2.35,5.26,7.87,5.83,7.45,3.96,3.96,3.97,3.97
73,2022-04-13 07:30:00,4.97,2.35,5.25,7.87,5.82,7.44,3.96,3.96,3.96,3.97
74,2022-04-13 07:45:00,4.96,2.34,5.26,7.86,5.81,7.43,3.95,3.96,3.96,3.96
75,2022-04-13 08:00:00,4.97,2.34,5.25,7.85,5.81,7.43,3.95,3.95,3.96,3.96
76,2022-04-13 08:15:00,4.96,2.34,5.25,7.84,5.81,7.42,3.95,3.95,3.95,3.96
77,2022-04-13 08:30:00,4.96,2.34,5.24,7.84,5.79,7.41,3.94,3.95,3.95,3.95
78,2022-04-13 08:45:00,4.95,2.34,5.24,7.83,5.80,7.40,3.94,3.94,3.95,3.95
79,2022-04-13 09:00:00,4.95,2.34,5.24,7.82,5.78,7.40,3.93,3.94,3.94,3.95
80,2022-04-13 09:15:00,4.95,2.33,5.23,7.82,5.78,7.40,3.93,3.93,3.94,3.94


In [6]:
df_train = df.sample(frac=0.8,random_state=200)
df_test = df.drop(df_train.index)

In [7]:
Y_train = df_train['LittleFallsGage']
X_train = df_train.drop(columns=['Date', 'LittleFallsGage'])

In [8]:
Y_test = df_test['LittleFallsGage']
X_test = df_test.drop(columns=['Date', 'LittleFallsGage'])

In [9]:
# need to break them out by seperate hourly predictions
#X_train_17hrs = X_train['SpringfieldGage']

In [10]:
X_train_17hrs = X_train[['SpringfieldGage','SpringfieldGage_extrashift1','SpringfieldGage_extrashift2','SpringfieldGage_extrashift3']].copy()

X_test_17hrs = X_test[['SpringfieldGage','SpringfieldGage_extrashift1','SpringfieldGage_extrashift2','SpringfieldGage_extrashift3']].copy()

In [11]:
#del(model_17hrs)
model_17hrs = tf.keras.Sequential([
    tf.keras.layers.Dense(8, activation = 'tanh', kernel_regularizer='l2', input_shape = (4,)),  # input shape required
    tf.keras.layers.Dense(10, activation = 'tanh', kernel_regularizer='l2'),
    tf.keras.layers.Dense(3, activation = 'tanh', kernel_regularizer='l2'),
    tf.keras.layers.Dense(1, kernel_regularizer='l2')
])

model_17hrs.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics = 'mean_squared_error')

2023-06-28 13:24:31.856221: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [12]:
# Train
model_17hrs.fit(X_train_17hrs , Y_train, epochs=150, verbose=0)

In [13]:
pred = model_17hrs.predict(X_test_17hrs)

620/620 [==============================] - 1s 799us/step


In [14]:
compare_df = pd.DataFrame(pred, columns=['Prediction'])
compare_df['TrueValue'] = Y_test.values
# used values bcuz...
# the index from the Y_test index is used to map it onto the new compare_df
# but the compare_df got its index form the prediction so it starts at 0 unlike Y_test
# so it only matches up rows correctly when the indexes match, which is few

In [15]:
compare_df.iloc[700:1000, :]

,Prediction,TrueValue
700,4.326269,4.43
701,4.323306,4.44
702,4.311455,4.41
703,4.305517,4.42
704,4.301051,4.42
705,4.295095,4.41
706,4.295095,4.41
707,4.295095,4.40
708,4.290631,4.40
709,4.289129,4.39


In [16]:
# not good enough
# need to add more features, like past 3 obs. 

In [17]:
# Save the model
model_17hrs.save('Models/model_17hrs.h5')

In [2]:
new_model = tf.keras.models.load_model('Models/model_17hrs.h5')

2023-07-06 15:22:04.191334: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:

# Convert into Numpy array
samples_to_predict = np.array(np.array([[5, 5, 5, 5], [4, 4, 4, 4]]))


new_model.predict(samples_to_predict)

1/1 [==============================] - 0s 37ms/step


array([[5.40041  ],
       [4.7992063]], dtype=float32)